# Импорт библиотек и подключение к Analytics_group

In [1]:
import pandas as pd
import re
import numpy as np
from tqdm.notebook import tqdm
import pyodbc
import os
import fpdf
from functools import reduce
import functools

import warnings
warnings.filterwarnings("ignore")

import sqlalchemy
from sqlalchemy import text
import pyodbc
from tqdm.notebook import tqdm
import time
import numpy as np
import random

import pandas as pd
import os
import glob 
import adodbapi
from time import sleep

conn_analutics_odbc_str = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=v-mktg-db;DATABASE=Analytics_group;Trusted_Connection=yes;"
def request_to_df(results, cursor):
    return pd.DataFrame([tuple(row) for row in results], columns=[column[0] for column in cursor.description])

# Чтение собранных продавцов 

In [3]:
sellers_info = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\промежуточный итог18.06.xlsx")
sellers_info

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,NaN
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,NaN
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,NaN
3,3,22301565032,https://www.wildberries.ru/seller/272815,SG fashion & accessory,Фэшен,361 529,NaN,NaN
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,NaN
...,...,...,...,...,...,...,...,...
2766,254,070711860657,https://www.wildberries.ru/seller/1402440,Adam,Парфюмерия,23 834,254.0,Индивидуальный предприниматель Баркал Гада
2767,255,165064455557,https://www.wildberries.ru/seller/4248052,Парфюмные хиты,Парфюмерия,4 860,255.0,Индивидуальный предприниматель Тилеков Чынтемир
2768,256,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,256.0,Индивидуальный предприниматель Богословская Лю...
2769,257,350705516274,https://www.wildberries.ru/seller/4025176,Perfume Dreams,Парфюмерия,41,257.0,Индивидуальный предприниматель Атакишиева Айгю...


# Получение данных из Analytics_group 

In [4]:
inns = list(sellers_info['ИНН'])
ip_inns = []
tot_df=[]
ul_inns = []
#print(inns)
for inn in inns:
    inn = str(inn)
    if len(inn) == 12:
        ip_inns.append(inn)
    elif len(inn) == 10:
        ul_inns.append(inn)

#print("ip: ", ip_inns)
#print("ul: ", ul_inns)

part_query = ' OR '.join(["inn = '" + inn + "'" for inn in ul_inns])
#print(part_query)


query = f'''
    SELECT inn, region, phone, status, email
    FROM SPARK_UL_info_v2
    WHERE ''' + part_query

query = query.strip()

conn_analytics = pyodbc.connect(conn_analutics_odbc_str)
cursor_analytics = conn_analytics.cursor()

cursor_analytics.execute(query)
results = cursor_analytics.fetchall()
df_part = request_to_df(results, cursor_analytics)
tot_df.append(df_part)

cursor_analytics.close()
conn_analytics.close()

df = pd.concat(tot_df, ignore_index=True)

part_query = ' OR '.join(["inn = '" + inn + "'" for inn in ip_inns])

query = f'''
    SELECT inn, region, phone, status, email
    FROM SPARK_IP_info_v2
    WHERE ''' + part_query

query = query.strip()

conn_analytics = pyodbc.connect(conn_analutics_odbc_str)
cursor_analytics = conn_analytics.cursor()

cursor_analytics.execute(query)
results = cursor_analytics.fetchall()
df_part = request_to_df(results, cursor_analytics)
tot_df.append(df_part)

cursor_analytics.close()
conn_analytics.close()

df1 = pd.concat(tot_df, ignore_index=True)
df = pd.concat([df, df1], axis = 0)
df

,inn,region,phone,status,email
0,9200017537,Севастополь,None,Действующая,kapitalgarantplus@yandex.ru
1,5405016008,Новосибирская область,"+7 (923) 1502220,+7 (983) 3032202,+7 (923) 112...",Действующая,"coffeeradosti@mail.ru, everybodyscrub@yandex.r..."
2,2100006306,Чувашская Республика-Чувашия,None,Действующая,olgagromilova@mail.ru
3,7713076301,Ульяновская область,"+7 (842) 2730055,+7 (842) 2730050",Действующая,None
4,7713076301,Архангельская область,+7 (818) 2657918,Действующая,None
...,...,...,...,...,...
2727,972708261010,Москва,None,Действующее,NATAPOGREBNYK@MAIL.RU
2728,972710036798,Москва,None,Действующее,SHAUCHENKAWB@GMAIL.COM
2729,972900710711,Москва,None,Действующее,A.IBRAGIMOVA721@GMAIL.COM
2730,972903391206,Москва,None,Действующее,PIROVTOHIR94@GMAIL.COM


# Объединение контактов в 1 столбец

In [7]:
df['Контакты'] = df['phone'].fillna('') + ' ' + df['email'].fillna('')
df['Контакты'] = df['Контакты'].replace('', None)
del df['phone']
del df['email']
df

,inn,region,status,Контакты
0,9200017537,Севастополь,Действующая,kapitalgarantplus@yandex.ru
1,5405016008,Новосибирская область,Действующая,"+7 (923) 1502220,+7 (983) 3032202,+7 (923) 112..."
2,2100006306,Чувашская Республика-Чувашия,Действующая,olgagromilova@mail.ru
3,7713076301,Ульяновская область,Действующая,"+7 (842) 2730055,+7 (842) 2730050"
4,7713076301,Архангельская область,Действующая,+7 (818) 2657918
...,...,...,...,...
2727,972708261010,Москва,Действующее,NATAPOGREBNYK@MAIL.RU
2728,972710036798,Москва,Действующее,SHAUCHENKAWB@GMAIL.COM
2729,972900710711,Москва,Действующее,A.IBRAGIMOVA721@GMAIL.COM
2730,972903391206,Москва,Действующее,PIROVTOHIR94@GMAIL.COM


In [8]:
#СОХРАНЯЕМ КАК ПРОМЕЖУТОЧНЫЙ РЕЗУЛЬТАТ
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из нашей бд18.06 с почтой.xlsx")

In [9]:
#ГОТОВИМ К СЛИЯНИЮ ИСХОДНЫЙ ДАТАСЕТ
df1 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\промежуточный итог18.06.xlsx")
df1['ИНН'] = df1['ИНН'].apply(str)
df1

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,NaN
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,NaN
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,NaN
3,3,22301565032,https://www.wildberries.ru/seller/272815,SG fashion & accessory,Фэшен,361 529,NaN,NaN
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,NaN
...,...,...,...,...,...,...,...,...
2766,254,070711860657,https://www.wildberries.ru/seller/1402440,Adam,Парфюмерия,23 834,254.0,Индивидуальный предприниматель Баркал Гада
2767,255,165064455557,https://www.wildberries.ru/seller/4248052,Парфюмные хиты,Парфюмерия,4 860,255.0,Индивидуальный предприниматель Тилеков Чынтемир
2768,256,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,256.0,Индивидуальный предприниматель Богословская Лю...
2769,257,350705516274,https://www.wildberries.ru/seller/4025176,Perfume Dreams,Парфюмерия,41,257.0,Индивидуальный предприниматель Атакишиева Айгю...


In [10]:
#ГОТОВИМ К СЛИЯНИЮ ДАТАСЕТ С КОНТАКТАМИ
df2 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из нашей бд18.06 с почтой.xlsx")
#df2 = df
df2 = df2.rename(columns={'inn': 'ИНН'})
df2['ИНН'] = df2['ИНН'].apply(str)
#df2.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Первая партия\из нашей бд.xlsx")
df2

,Unnamed: 0,ИНН,region,status,Контакты
0,0,9200017537,Севастополь,Действующая,kapitalgarantplus@yandex.ru
1,1,5405016008,Новосибирская область,Действующая,"+7 (923) 1502220,+7 (983) 3032202,+7 (923) 112..."
2,2,2100006306,Чувашская Республика-Чувашия,Действующая,olgagromilova@mail.ru
3,3,7713076301,Ульяновская область,Действующая,"+7 (842) 2730055,+7 (842) 2730050"
4,4,7713076301,Архангельская область,Действующая,+7 (818) 2657918
...,...,...,...,...,...
3425,2727,972708261010,Москва,Действующее,NATAPOGREBNYK@MAIL.RU
3426,2728,972710036798,Москва,Действующее,SHAUCHENKAWB@GMAIL.COM
3427,2729,972900710711,Москва,Действующее,A.IBRAGIMOVA721@GMAIL.COM
3428,2730,972903391206,Москва,Действующее,PIROVTOHIR94@GMAIL.COM


# Соединение данных

In [11]:
df = pd.merge(df1, df2, on = 'ИНН', how = 'left')
df

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0_x,Официальное название,Unnamed: 0_y,region,status,Контакты
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,NaN,1999.0,Московская область,Действующее,CHUMAKOVA_VA@MAIL.RU
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,NaN,1192.0,Свердловская область,Действующее,MALLAEV.AMIR@BK.RU
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,NaN,1719.0,Архангельская область,Действующее,DMITRY_AKSENOV@HOTMAIL.COM
3,3,22301565032,https://www.wildberries.ru/seller/272815,SG fashion & accessory,Фэшен,361 529,NaN,NaN,NaN,NaN,NaN,NaN
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,NaN,2530.0,Москва,Действующее,SMART-HUNTER@YANDEX.RU
...,...,...,...,...,...,...,...,...,...,...,...,...
4168,254,070711860657,https://www.wildberries.ru/seller/1402440,Adam,Парфюмерия,23 834,254.0,Индивидуальный предприниматель Баркал Гада,NaN,NaN,NaN,NaN
4169,255,165064455557,https://www.wildberries.ru/seller/4248052,Парфюмные хиты,Парфюмерия,4 860,255.0,Индивидуальный предприниматель Тилеков Чынтемир,NaN,NaN,NaN,NaN
4170,256,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,256.0,Индивидуальный предприниматель Богословская Лю...,2134.0,Москва,Действующее,TEENKA@YANDEX.RU
4171,257,350705516274,https://www.wildberries.ru/seller/4025176,Perfume Dreams,Парфюмерия,41,257.0,Индивидуальный предприниматель Атакишиева Айгю...,1828.0,Вологодская область,Действующее,EVEREST0505@MAIL.RU


In [12]:
#СОХРАНЯЕМ ПОЛУЧИВШЕЕСЯ
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\собранное+из бд с почтой.xlsx")

In [13]:
#ПОДЧИЩАЕМ КОНТАКТЫ
df = df.dropna(subset = ['Контакты'])
df = df.drop_duplicates(subset = ['Контакты'])
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\очищенный с почтой.xlsx")
df

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0_x,Официальное название,Unnamed: 0_y,region,status,Контакты
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,NaN,1999.0,Московская область,Действующее,CHUMAKOVA_VA@MAIL.RU
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,NaN,1192.0,Свердловская область,Действующее,MALLAEV.AMIR@BK.RU
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,NaN,1719.0,Архангельская область,Действующее,DMITRY_AKSENOV@HOTMAIL.COM
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,NaN,2530.0,Москва,Действующее,SMART-HUNTER@YANDEX.RU
5,5,771577539871,https://www.wildberries.ru/seller/55034,XELA,Фэшен,507 689,NaN,NaN,2328.0,Мурманская область,Действующее,COKOLOVCKI@GMAIL.COM
...,...,...,...,...,...,...,...,...,...,...,...,...
4155,244,231136188300,https://www.wildberries.ru/seller/669142,SHAIKH PARFUM,Парфюмерия,93 668,244.0,Индивидуальный предприниматель Оруджова Шабнам...,1540.0,Волгоградская область,Действующее,SOLTANOV_RASIM@INBOX.RU
4158,247,581304781867,https://www.wildberries.ru/seller/183599,MARKETNA,Парфюмерия,22 365,247.0,Индивидуальный предприниматель Лебедева Натали...,2033.0,Московская область,Действующее,ANDRIYANOVA_93@INBOX.RU
4161,249,551405483106,https://www.wildberries.ru/seller/1420874,ROSE BREND,Парфюмерия,22 826,249.0,ИП,1980.0,Москва,Действующее,AINAGUL-78@MAIL.RU
4170,256,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,256.0,Индивидуальный предприниматель Богословская Лю...,2134.0,Москва,Действующее,TEENKA@YANDEX.RU


In [14]:
#ОЧИЩАЕМ ФИЛИАЛЫ ОДНОЙ И ТОЙ ЖЕ КОМПАНИИ, НАХОДЯЩИЕСЯ В ОДНОМ РЕГИОНЕ
df = df.drop_duplicates(subset = ['ИНН', 'region'])
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\С усеченными филиалами из 1 региона исх.xlsx")
df

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0_x,Официальное название,Unnamed: 0_y,region,status,Контакты
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,NaN,1999.0,Московская область,Действующее,CHUMAKOVA_VA@MAIL.RU
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,NaN,1192.0,Свердловская область,Действующее,MALLAEV.AMIR@BK.RU
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,NaN,1719.0,Архангельская область,Действующее,DMITRY_AKSENOV@HOTMAIL.COM
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,NaN,2530.0,Москва,Действующее,SMART-HUNTER@YANDEX.RU
5,5,771577539871,https://www.wildberries.ru/seller/55034,XELA,Фэшен,507 689,NaN,NaN,2328.0,Мурманская область,Действующее,COKOLOVCKI@GMAIL.COM
...,...,...,...,...,...,...,...,...,...,...,...,...
4155,244,231136188300,https://www.wildberries.ru/seller/669142,SHAIKH PARFUM,Парфюмерия,93 668,244.0,Индивидуальный предприниматель Оруджова Шабнам...,1540.0,Волгоградская область,Действующее,SOLTANOV_RASIM@INBOX.RU
4158,247,581304781867,https://www.wildberries.ru/seller/183599,MARKETNA,Парфюмерия,22 365,247.0,Индивидуальный предприниматель Лебедева Натали...,2033.0,Московская область,Действующее,ANDRIYANOVA_93@INBOX.RU
4161,249,551405483106,https://www.wildberries.ru/seller/1420874,ROSE BREND,Парфюмерия,22 826,249.0,ИП,1980.0,Москва,Действующее,AINAGUL-78@MAIL.RU
4170,256,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,256.0,Индивидуальный предприниматель Богословская Лю...,2134.0,Москва,Действующее,TEENKA@YANDEX.RU
